---
---

# Cleaning, Casting & Merging

---
---

In [1]:
import pandas as pd
import numpy as np
import fastparquet as fpq
import re

---

# Ratebeer

---

In [2]:
df_rb = pd.read_parquet("./ratebeer.parquet", engine="fastparquet")

In [3]:
df_rb

,beer/name,beer/beerId,beer/brewerId,beer/ABV,beer/style,review/appearance,review/aroma,review/palate,review/taste,review/overall,review/time,review/profileName,review/text
0,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale &#40;IPA&#41;,4/5,6/10,3/5,6/10,13/20,1157587200,hopdog,"On tap at the Springfield, PA location. Poured..."
1,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale &#40;IPA&#41;,4/5,6/10,4/5,7/10,13/20,1157241600,TomDecapolis,On tap at the John Harvards in Springfield PA....
2,John Harvards Cristal Pilsner,71716,8481,5,Bohemian Pilsener,4/5,5/10,3/5,6/10,14/20,958694400,PhillyBeer2112,"UPDATED: FEB 19, 2003 Springfield, PA. I've ne..."
3,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Klsch,2/5,4/10,2/5,4/10,8/20,1157587200,TomDecapolis,On tap the Springfield PA location billed as t...
4,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Klsch,2/5,4/10,2/5,4/10,8/20,1157587200,hopdog,"On tap at the Springfield, PA location. Poured..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924158,Oakhill Mendip Twister,12240,1028,6.3,Stout,3/5,7/10,3/5,7/10,15/20,1013731200,rauchbier,"Doesnt suggest it from the bottle label, but f..."
2924159,Oakhill Yeoman 1767 Strong Ale,5677,1028,5,Premium Bitter/ESB,4/5,5/10,3/5,6/10,12/20,1074816000,imdownthepub,"Cask conditioned at The Wild Goose, Combeignte..."
2924160,Oakhill Yeoman 1767 Strong Ale,5677,1028,5,Premium Bitter/ESB,4/5,4/10,3/5,6/10,13/20,1050796800,SilkTork,The original Oakhill Brewery was formed in 176...
2924161,Oakhill Yeoman 1767 Strong Ale,5677,1028,5,Premium Bitter/ESB,3/5,5/10,2/5,5/10,10/20,1016409600,omhper,Sampled cask conditioned. Amber coloured. Frui...


In [4]:
for col in ("review/appearance","review/aroma","review/palate","review/taste","review/overall"):
    df_rb[col] = df_rb[col].apply(lambda x: float(x.split("/")[0]))

In [5]:

#! Scaling all the features to range 0-5

df_rb['review/aroma'] /= 2
df_rb['review/taste'] /= 2
df_rb['review/overall'] /= 4

In [6]:

#! Converting ABV to numberic and filling non-numeric values with NaN
df_rb['beer/ABV'] = pd.to_numeric(df_rb['beer/ABV'], errors='coerce')
df_rb['review/time'] = pd.to_datetime(df_rb['review/time'].astype(np.int64), unit='s', cache=True)
df_rb['review/site'] = "RateBeer"


In [7]:
df_rb.describe()

,beer/ABV,review/appearance,review/aroma,review/palate,review/taste,review/overall,review/time
count,2.785526e+06,2.924163e+06,2.924163e+06,2.924163e+06,2.924163e+06,2.924163e+06,2924163
mean,6.640794e+00,3.432027e+00,3.176804e+00,3.257236e+00,3.225964e+00,3.299924e+00,2008-03-08 16:15:45.885438976
min,1.000000e-02,1.000000e+00,5.000000e-01,1.000000e+00,5.000000e-01,2.500000e-01,2000-04-12 00:00:00
25%,5.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,2006-05-24 00:00:00
50%,6.000000e+00,3.000000e+00,3.500000e+00,3.000000e+00,3.500000e+00,3.500000e+00,2008-07-08 00:00:00
75%,8.000000e+00,4.000000e+00,3.500000e+00,4.000000e+00,4.000000e+00,3.750000e+00,2010-05-09 00:00:00
max,5.770000e+01,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,2012-01-13 00:00:00
std,2.313753e+00,8.125569e-01,8.195824e-01,8.297312e-01,8.114442e-01,8.381448e-01,NaN


In [8]:
df_rb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924163 entries, 0 to 2924162
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   beer/name           object        
 1   beer/beerId         object        
 2   beer/brewerId       object        
 3   beer/ABV            float64       
 4   beer/style          object        
 5   review/appearance   float64       
 6   review/aroma        float64       
 7   review/palate       float64       
 8   review/taste        float64       
 9   review/overall      float64       
 10  review/time         datetime64[ns]
 11  review/profileName  object        
 12  review/text         object        
 13  review/site         object        
dtypes: datetime64[ns](1), float64(6), object(7)
memory usage: 312.3+ MB


In [9]:
(df_rb.count()/len(df_rb)*100).sort_values(ascending=True)

beer/ABV               95.258917
beer/name             100.000000
beer/beerId           100.000000
beer/brewerId         100.000000
beer/style            100.000000
review/appearance     100.000000
review/aroma          100.000000
review/palate         100.000000
review/taste          100.000000
review/overall        100.000000
review/time           100.000000
review/profileName    100.000000
review/text           100.000000
review/site           100.000000
dtype: float64

In [10]:
df_rb.to_parquet("./ratebeer_clean.parquet", engine="fastparquet")


---

# Beeradvocate

---

In [11]:
df_ba= pd.read_parquet("./beeradvocate.parquet", engine="fastparquet")

In [12]:
df_ba.describe()

,beer/name,beer/beerId,beer/brewerId,beer/ABV,beer/style,review/appearance,review/aroma,review/palate,review/taste,review/overall,review/time,review/profileName,review/text
count,1586614,1586614,1586614,1586614,1586614,1586614,1586614,1586614,1586614,1586614,1586614,1586614,1586614
unique,56857,66055,5840,531,104,10,9,9,9,10,1577960,33388,1585261
top,90 Minute IPA,2093,35,5.00,American IPA,4,4,4,4,4,1101772800,northyorksammy,
freq,3290,3290,39444,109144,117586,674186,557383,606711,541429,582764,21,5817,353


In [13]:
df_ba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   beer/name           1586614 non-null  object
 1   beer/beerId         1586614 non-null  object
 2   beer/brewerId       1586614 non-null  object
 3   beer/ABV            1586614 non-null  object
 4   beer/style          1586614 non-null  object
 5   review/appearance   1586614 non-null  object
 6   review/aroma        1586614 non-null  object
 7   review/palate       1586614 non-null  object
 8   review/taste        1586614 non-null  object
 9   review/overall      1586614 non-null  object
 10  review/time         1586614 non-null  object
 11  review/profileName  1586614 non-null  object
 12  review/text         1586614 non-null  object
dtypes: object(13)
memory usage: 157.4+ MB


In [14]:
df_ba['beer/ABV'] = pd.to_numeric(df_ba['beer/ABV'], errors='coerce')
df_ba['review/time'] = pd.to_datetime(df_ba['review/time'].astype(np.int64), unit='s', cache=True)
df_ba['review/site'] = "BeerAdvocate"

In [15]:
for col in ("review/appearance","review/aroma","review/palate","review/taste","review/overall"):
    df_ba[col] = df_ba[col].astype(np.float64)

In [16]:
df_ba.describe()

,beer/ABV,review/appearance,review/aroma,review/palate,review/taste,review/overall,review/time
count,1.518829e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1586614
mean,7.042387e+00,3.841642e+00,3.735636e+00,3.743701e+00,3.792860e+00,3.815581e+00,2008-10-15 16:48:00.012210944
min,1.000000e-02,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1996-08-22 00:00:01
25%,5.200000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,2007-03-06 23:36:28.249999872
50%,6.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,2009-04-08 15:01:21.500000
75%,8.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,4.500000e+00,2010-10-31 23:40:05.249999872
max,5.770000e+01,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,2012-01-11 12:35:48
std,2.322526e+00,6.160928e-01,6.976167e-01,6.822184e-01,7.319696e-01,7.206219e-01,NaN


In [17]:
df_ba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   beer/name           1586614 non-null  object        
 1   beer/beerId         1586614 non-null  object        
 2   beer/brewerId       1586614 non-null  object        
 3   beer/ABV            1518829 non-null  float64       
 4   beer/style          1586614 non-null  object        
 5   review/appearance   1586614 non-null  float64       
 6   review/aroma        1586614 non-null  float64       
 7   review/palate       1586614 non-null  float64       
 8   review/taste        1586614 non-null  float64       
 9   review/overall      1586614 non-null  float64       
 10  review/time         1586614 non-null  datetime64[ns]
 11  review/profileName  1586614 non-null  object        
 12  review/text         1586614 non-null  object        
 13  review/site 

In [18]:
(df_ba.count()/len(df_ba)*100).sort_values(ascending=True)

beer/ABV               95.727694
beer/name             100.000000
beer/beerId           100.000000
beer/brewerId         100.000000
beer/style            100.000000
review/appearance     100.000000
review/aroma          100.000000
review/palate         100.000000
review/taste          100.000000
review/overall        100.000000
review/time           100.000000
review/profileName    100.000000
review/text           100.000000
review/site           100.000000
dtype: float64

In [19]:
df_ba.to_parquet("./beeradvocate_clean.parquet", engine="fastparquet")


---

## Combining the two

---

In [20]:
df = pd.concat([df_rb, df_ba], axis=0, ignore_index=True)

In [21]:
np.round(df.describe(), decimals=3)

,beer/ABV,review/appearance,review/aroma,review/palate,review/taste,review/overall,review/time
count,4304355.000,4510777.000,4510777.000,4510777.000,4510777.000,4510777.000,4510777
mean,6.782,3.576,3.373,3.428,3.425,3.481,2008-05-25 10:04:22.078417920
min,0.010,0.000,0.500,1.000,0.500,0.000,1996-08-22 00:00:01
25%,5.000,3.000,3.000,3.000,3.000,3.000,2006-08-30 00:00:00
50%,6.000,4.000,3.500,3.500,3.500,3.500,2008-10-21 23:04:11
75%,8.000,4.000,4.000,4.000,4.000,4.000,2010-07-23 00:00:00
max,57.700,5.000,5.000,5.000,5.000,5.000,2012-01-13 00:00:00
std,2.325,0.774,0.823,0.815,0.830,0.836,NaN


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510777 entries, 0 to 4510776
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   beer/name           object        
 1   beer/beerId         object        
 2   beer/brewerId       object        
 3   beer/ABV            float64       
 4   beer/style          object        
 5   review/appearance   float64       
 6   review/aroma        float64       
 7   review/palate       float64       
 8   review/taste        float64       
 9   review/overall      float64       
 10  review/time         datetime64[ns]
 11  review/profileName  object        
 12  review/text         object        
 13  review/site         object        
dtypes: datetime64[ns](1), float64(6), object(7)
memory usage: 481.8+ MB


In [23]:

#! Percentage of non-null values in each column

(df.count()/len(df)*100).sort_values(ascending=True)

beer/ABV               95.423804
beer/name             100.000000
beer/beerId           100.000000
beer/brewerId         100.000000
beer/style            100.000000
review/appearance     100.000000
review/aroma          100.000000
review/palate         100.000000
review/taste          100.000000
review/overall        100.000000
review/time           100.000000
review/profileName    100.000000
review/text           100.000000
review/site           100.000000
dtype: float64

In [24]:
df[df['beer/ABV'].isna()]

,beer/name,beer/beerId,beer/brewerId,beer/ABV,beer/style,review/appearance,review/aroma,review/palate,review/taste,review/overall,review/time,review/profileName,review/text,review/site
5,John Harvards Vanilla Black Velvet Stout,31544,8481,NaN,Sweet Stout,5.0,4.0,4.0,3.5,4.00,2004-02-26 00:00:00,egajdzis,"Springfield, PA location... Poured an opaque b...",RateBeer
6,John Harvards American Brown Ale,71714,8481,NaN,Brown Ale,4.0,2.5,3.0,3.0,3.00,2007-04-09 00:00:00,hopdog,"On tap at the Springfield, PA location. Listed...",RateBeer
11,John Harvards Cassis Wit,1470,8481,NaN,Belgian White &#40;Witbier&#41;,3.0,2.5,3.0,3.5,3.50,2000-05-19 00:00:00,PhillyBeer2112,"UPDATED: FEB 19, 2003 Springfield, PA. Interes...",RateBeer
23,Barley Island Dirty &quot;Old&quot; Helen Sour...,114513,3228,NaN,Sour Ale/Wild Ale,4.0,3.5,3.0,3.0,3.25,2010-05-14 00:00:00,fiulijn,"Courtesy of Kim, handbottledCloudy amber color...",RateBeer
24,Barley Island Dirty &quot;Old&quot; Helen Sour...,114513,3228,NaN,Sour Ale/Wild Ale,4.0,3.5,3.0,3.0,3.25,2010-05-03 00:00:00,Ungstrup,Handbottled. A dark amber beer with a white he...,RateBeer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4510749,Brandy Brown,45508,14359,NaN,American Brown Ale,4.0,3.5,2.5,2.5,3.00,2008-10-21 17:22:51,kasper,Notes from cask sampled at Blind Tiger in May....,BeerAdvocate
4510750,Maibock,36555,14359,NaN,Maibock / Helles Bock,4.0,4.5,4.5,4.0,3.50,2007-04-29 10:22:48,BBM,I hadn't heard of Defiant before trying this b...,BeerAdvocate
4510759,Resolution #2,48360,14359,NaN,Belgian Strong Pale Ale,5.0,3.0,4.0,3.5,4.00,2010-10-24 20:11:07,hoppymcgee,"Had on tap at Bierkraft in Brooklyn, NY. Poure...",BeerAdvocate
4510760,Resolution #2,48360,14359,NaN,Belgian Strong Pale Ale,4.0,4.5,4.0,4.0,4.50,2009-05-09 21:57:03,WesWes,The beer pours a hazy gold color with a thick ...,BeerAdvocate


### Removing non english & gibberish reviews

```Python

# Code was executed on Kaggle for 4 hours
# Removing non-english & gibberish reviews

from langdetect import detect, lang_detect_exception
from time import time
non_eng = []
gibberish = []
n = len(df)
count = 0
start = time()
for index, review in enumerate(df['review/text']):
    tr = round((time()-start)*n/((index+1)*60),4)
    try:
        lang = detect(review) 
    except lang_detect_exception.LangDetectException:
        gibberish.append((index,review))
        count +=1
        print("GIBBERISH: <start>", review, "<end>")
        continue
    if  lang != 'en':
        non_eng.append((index,review))
        count += 1
        print(f"ET: {tr:>8} mins |", lang, f"| [{index}]",f"[{round(count/index*100,3)}%]", review[:150] )
```


In [25]:
with open("./utils/useless_non_eng", "r") as f:
    non_eng = [int(i) for i in f.readlines()]
with open("./utils/useless_gibberish", "r") as f:
    gibberish = [int(i) for i in f.readlines()]

useless_index = non_eng + gibberish

In [26]:
len(useless_index)

46288

In [27]:
df = df.drop(useless_index).reset_index(drop=True)

In [29]:
df

,beer/name,beer/beerId,beer/brewerId,beer/ABV,beer/style,review/appearance,review/aroma,review/palate,review/taste,review/overall,review/time,review/profileName,review/text,review/site,review/id
0,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale &#40;IPA&#41;,4.0,3.0,3.0,3.0,3.25,2006-09-07 00:00:00,hopdog,"On tap at the Springfield, PA location. Poured...",RateBeer,review_0
1,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale &#40;IPA&#41;,4.0,3.0,4.0,3.5,3.25,2006-09-03 00:00:00,TomDecapolis,On tap at the John Harvards in Springfield PA....,RateBeer,review_1
2,John Harvards Cristal Pilsner,71716,8481,5.0,Bohemian Pilsener,4.0,2.5,3.0,3.0,3.50,2000-05-19 00:00:00,PhillyBeer2112,"UPDATED: FEB 19, 2003 Springfield, PA. I've ne...",RateBeer,review_2
3,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Klsch,2.0,2.0,2.0,2.0,2.00,2006-09-07 00:00:00,TomDecapolis,On tap the Springfield PA location billed as t...,RateBeer,review_3
4,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Klsch,2.0,2.0,2.0,2.0,2.00,2006-09-07 00:00:00,hopdog,"On tap at the Springfield, PA location. Poured...",RateBeer,review_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4464484,The Horseman's Ale,33061,14359,5.2,Pumpkin Ale,3.5,4.0,4.0,4.0,5.00,2006-11-05 00:01:32,maddogruss,If you're looking for a quality beer right alo...,BeerAdvocate,review_4464484
4464485,The Horseman's Ale,33061,14359,5.2,Pumpkin Ale,2.5,5.0,2.0,4.0,4.00,2006-10-17 01:29:26,yelterdow,Growler fill from the long-awaited Defiant Bre...,BeerAdvocate,review_4464485
4464486,The Horseman's Ale,33061,14359,5.2,Pumpkin Ale,3.0,3.5,3.5,4.0,4.50,2006-10-13 01:21:53,TongoRad,This was served from a growler and I suspect a...,BeerAdvocate,review_4464486
4464487,The Horseman's Ale,33061,14359,5.2,Pumpkin Ale,4.5,4.5,4.5,4.5,4.00,2006-10-05 04:37:24,dherling,"Ah, the horseman commeth. Cloudy chocolate bro...",BeerAdvocate,review_4464487


In [30]:
df['review/text'] = df['review/text'].apply(lambda text: re.sub(r'\s+', ' ', text))

In [ ]:
df.to_parquet("./combined.parquet", engine="fastparquet", times="int96")